# RSA+OAEP vs ElGamal vs ECC-ElGamal Benchmarks

- Device: Dell XPS 15 9570 2018
- OS: UBUNTU 20.04 (stable)
- Memory: 16GiB System Memory, 2x 8GiB SODIMM DDR4 Synchronous 2667 MHz
- Cache: L1 cache -> 384KiB, L2 cache -> 1536KiB, L3 cache -> 9Mib
- Processor: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, 6 Cores, 12 threads
- Graphic Card: Integrated -> UHD Graphics 630 Mobile, Discrete -> GeForce GTX 1050 Ti Mobile

## Imports

In [1]:
import sys
import time
import warnings
from pathlib import Path

path = Path.cwd().resolve()
warnings.filterwarnings("ignore")

In [2]:
import plotly.graph_objects as px
from pyinstrument import Profiler as pyProfiler

pyProfiler = pyProfiler()

import psutil
from pyJoules.device import DeviceFactory
from pyJoules.device.rapl_device import RaplCoreDomain
from pyJoules.energy_meter import EnergyMeter

device = DeviceFactory.create_devices([RaplCoreDomain(0)])
meter = EnergyMeter(device)

%load_ext memory_profiler

In [3]:
from rsa import rsa

sys.path.append("/home/apollo/Desktop/DJSCE/Projects/css_research/ecc_elgamal")
from ecc_elgamal import elgamal as ecc_elg
from ecc_elgamal.cipher.curve import (E222, E382, M383, P256, Curve25519,
                                      secp256k1)

sys.path.append("/home/apollo/Desktop/DJSCE/Projects/css_research/norm_elgamal")

from norm_elgamal import elgamal as norm_elg

In [4]:
wall_time, mem_use, cpu_use, power_consumed, similarity = [], [], [], [], []

key_sizes = [1024, 2048, 3072, 4096]
message_sizes = [1, 8, 64, 512, 4096]
messages = [
    b"At.",
    b"Vivamus.",
    b"Lorem ipsum dolor sit amet, consectetur adipiscing elit vivamus.",
    b"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed eu nibh accumsan, pretium lorem sed, fermentum eros. Proin mi massa, hendrerit et bibendum vel, pulvinar eu turpis. Donec vel gravida ipsum, in vehicula odio. Integer non mi tincidunt, pulvinar urna id, gravida ante. Nullam accumsan ac odio eget bibendum. Cras rhoncus turpis et orci imperdiet, facilisis facilisis purus dignissim. Cras pretium sapien sit amet velit condimentum eleifend. Vestibulum consequat metus neque, nec pretium ex tincidunt at.",
    b"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer dignissim eros velit, tristique ultrices sem vestibulum id. Nunc sit amet cursus nulla. Morbi eget nibh dolor. Vestibulum mattis purus nec blandit tristique. Ut libero lacus, vestibulum nec imperdiet vel, ullamcorper et dolor. Quisque fringilla interdum pulvinar. Nulla eleifend nisi quam, ac pellentesque metus blandit in. Suspendisse potenti. Vivamus mattis neque ultricies, tincidunt neque vel, vulputate orci. Morbi ultrices bibendum enim ac vehicula. Mauris in luctus mauris, sit amet lacinia dui. Mauris in urna a est sagittis pretium quis consequat erat. Maecenas tincidunt convallis sem elementum auctor. Morbi arcu nibh, semper eget sollicitudin a, vestibulum sed erat. Quisque posuere magna vitae massa sodales dignissim. Nullam finibus, neque mattis pellentesque faucibus, mi orci iaculis arcu, vitae venenatis elit libero vitae sem. Phasellus elementum neque sit amet nibh condimentum malesuada. Morbi fermentum sapien dui, eget posuere lorem eleifend non. Pellentesque a diam lacinia tellus rutrum tincidunt vel tristique turpis. Sed tempor scelerisque odio, eu scelerisque mi tincidunt ac. Sed pulvinar leo vel massa volutpat, vitae dignissim lacus condimentum. Aenean elementum urna ac magna iaculis dapibus. Quisque libero leo, mollis et neque quis, scelerisque tincidunt ex. In facilisis leo lacus, sit amet viverra metus auctor vel. Sed bibendum porttitor accumsan. Duis iaculis dignissim enim, vitae ornare enim. Phasellus quis blandit nisi. Curabitur at turpis nec velit molestie mollis id in massa. Maecenas laoreet cursus nibh volutpat accumsan. Donec cursus tristique neque, at vulputate mauris posuere sit amet. Vestibulum eu leo at augue hendrerit malesuada non vitae erat. Quisque justo nunc, consectetur et rutrum et, lacinia commodo dui. Aenean eget eros a metus sollicitudin vestibulum. Nam convallis turpis augue, in varius libero iaculis nec. In tincidunt neque ac neque rhoncus tristique. In a tellus feugiat, tincidunt elit eget, pulvinar nibh. Nullam in nisi nulla. Duis iaculis, ante at dapibus vulputate, lorem massa venenatis ipsum, ut eleifend est leo vitae sem. Ut faucibus eros ut tincidunt vulputate. Donec rhoncus, enim sed imperdiet vestibulum, augue ante vulputate ante, sit amet ornare eros nisi ac velit. Curabitur finibus consectetur velit, vel viverra risus vestibulum sagittis. Praesent pellentesque nulla et ipsum mattis, quis sodales ante rutrum. Aliquam erat volutpat. Interdum et malesuada fames ac ante ipsum primis in faucibus. Vestibulum vitae urna id mauris facilisis ultricies. Proin tempor libero nulla, ac vestibulum augue tincidunt at. In aliquet, nisl vel eleifend fermentum, ligula tortor fermentum lorem, in malesuada ex nunc at mauris. Sed ac auctor enim. Nunc placerat convallis urna interdum commodo. Nulla facilisi. Fusce quis mattis nibh. Sed porttitor placerat bibendum. Praesent aliquet nisl ac nibh interdum molestie. Maecenas non magna a sem porttitor porta sed sed augue. Phasellus eu vulputate est, quis ultricies leo. Vivamus condimentum nec neque sed finibus. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Fusce eget arcu lorem. Phasellus vestibulum porta sapien tincidunt congue. Morbi in vehicula ante. Nam eu fringilla augue. Curabitur posuere, orci vitae consequat placerat, risus est condimentum tortor, sed vulputate diam justo in lacus. Sed tempor, magna et finibus aliquam, mauris leo efficitur felis, eget laoreet erat ex quis libero. Cras luctus, elit non scelerisque dapibus, purus dui suscipit tellus, vel posuere ligula velit ac sem. Vivamus tincidunt mauris nec iaculis luctus. Integer venenatis erat in nisi tempor pretium. Proin nec consequat felis. Aliquam interdum non enim non eleifend. Phasellus at nulla blandit eros lacinia tempor sit amet euismod nisi. Mauris ut maximus libero. Fusce tristique tincidunt maximus. Nullam dignissim scelerisque mi eget ultricies. Donec lectus velit, hendrerit et dolor fringilla, rutrum aliquet risus. In aliquam quam et porta pharetra. In ut leo quis dolor orci.",
]

<br>

## Testing Helper Functions

In [5]:
def doNothing(tp_time):
    time.sleep(tp_time)

In [6]:
# Similarity Matching using Damerau Levenshtein Distance
def textSimilarity(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)

    for i in range(-1, lenstr1 + 1):
        d[(i, -1)] = i + 1

    for j in range(-1, lenstr2 + 1):
        d[(-1, j)] = j + 1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1

            d[(i, j)] = min(
                d[(i - 1, j)] + 1,  # deletion
                d[(i, j - 1)] + 1,  # insertion
                d[(i - 1, j - 1)] + cost,  # substitution
            )

            if i and j and s1[i] == s2[j - 1] and s1[i - 1] == s2[j]:
                d[(i, j)] = min(d[(i, j)], d[i - 2, j - 2] + cost)  # transposition

    return round(1 - ((d[lenstr1-1,lenstr2-1])/(lenstr1 if lenstr1 > lenstr2 else lenstr2)), 4)

In [7]:
def drawGraph(func: list, graph_vals: list, name: str) -> None:
    """
    Plots graph for function.

    Parameter
    ----------
    func : list
        Values obtained for params
    graph_vals : list
        Measurement values
    name : str
        Placeholder for file name
    """
    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=("Execution Time(in s)(log)", "Memory Use(in MB)(log)"),
    )

    fig.add_trace(px.Scatter(x=graph_val, y=wall_time, name="Wall Time"), row=1, col=1)
    fig.update_yaxes(type="log", row=1, col=1)

    fig.add_trace(px.Scatter(x=graph_val, y=mem_use, name="Memory Used"), row=1, col=2)
    fig.update_yaxes(type="log", row=1, col=2)

    fig.update_layout(title_text=f"Benchmark for {name}")

    fig.write_html(f"./{name}/{name}_plotly.html")
    fig.show()

    for i in range(0, len(wall_time)):
        print(
            f"Counts: {graph_val[i]}, Time: {round(wall_time[i], 3)} s, Memory: {round(mem_use[i], 3)} MiB"
        )

    func.clear()

In [8]:
def benchCode(benchFunc, *args):
    """
    Generalized code for benchmarking cryptographic functions.

    Parameter
    ----------
    benchFunc : function
        The function to be benchmarked.
    """
    curr_time = %timeit -r 5 -o -q -c -p 5 benchFunc(*args)
    print(curr_time)
    doNothing(5)
    curr_mem = %memit -o -q benchFunc(*args)
    print(curr_mem)
    doNothing(5)
    doNothing(5)
    curr_cpu_use = psutil.getloadavg()[0] / psutil.cpu_count() * 100
    print(curr_cpu_use)
    # meter.start(tag='foo')
    # doNothing(5)
    # doNothing(5)
    # meter.record(tag='bar')
    # benchFunc(*args)
    # meter.stop()
    # curr_power = meter.get_trace()

    plain_cipher_sim = textSimilarity(plain_text, cipher_text)
    print(plain_cipher_sim)
    # wall_time.append(time1.average)
    # mem_use.append(mem1.mem_usage[0])

    # del mem1, time1

<br>

## RSA with OAEP

In [9]:
name = "rsa/RSA_IP"

try:
    dir_path = path.joinpath(name)
    dir_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    pass

In [10]:
for key in key_sizes:
    for plain_text in messages:
        try:
            benchCode(rsa.caller, plain_text, key)
        except ValueError:
            pass
            

# drawGraph(wall_time, mem_use, name)

93.135 ms ± 18.306 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
peak memory: 77.31 MiB, increment: 0.38 MiB
8.833333333333334


NameError: name 'cipher_text' is not defined